In [8]:
import pandas as pd
import apyori



In [ ]:
df = pd.read_csv("./data/netflix/data_fin.csv", sep=";")


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   int64  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int64(2)
memory usage: 2.2 GB


In [3]:
good = (
    df[df["Rating"] == 5]
    .groupby("Cust_Id")["Movie_Id"]
    .apply(lambda r: " ".join([str(A) for A in r]))
)


In [7]:
from pprint import pprint
pprint(good)

Cust_Id
6          175 457 886 1467 2372 2452 2782 3290 4043 4633...
7          8 30 83 175 257 283 285 313 357 457 458 468 50...
8          1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10         473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                     4432 6786 7605 9326 10643 15107 15270
                                 ...                        
2649404    550 2174 2200 3427 3936 4996 6972 9340 10378 1...
2649409    2913 4951 5327 10229 11077 11722 11812 14103 1...
2649421    1877 2152 3078 4640 5317 6287 7617 8438 9756 1...
2649426    197 273 406 468 937 1073 1220 1803 1905 2457 2...
2649429    30 143 199 533 607 759 1798 1799 1962 1975 209...
Name: Movie_Id, Length: 463616, dtype: object


In [19]:
association_rules = apyori.apriori(
    good.apply(lambda r: r.split(" ")),
    min_support=0.04,
    min_confidence=0.1,
    min_lift=2,
    min_length=2,
)

In [10]:
asr_df = pd.DataFrame(columns=["from", "to", "confidence", "support", "lift"])
for item in association_rules:
    pair = item[0]
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] = (
        " ".join(list(item[2][0][0])),
        " ".join(list(item[2][0][1])),
        item[2][0][2],
        item[1],
        item[2][0][3],
    )
asr_df.head()

In [11]:
titles = pd.read_csv(
    "./data/netflix/movie_titles.csv",
    encoding="ISO-8859-1",
    header=None,
    names=["Movie_Id", "Year", "Name"],
)


In [12]:
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule["from"].split(" "))]["Name"].values)
    print("----------->")
    print(titles[titles.Movie_Id == int(rule["to"])]["Name"].values)


In [13]:
get_rule_title(asr_df.loc[99])


['Monsters']
----------->
['Finding Nemo (Widescreen)']


In [14]:
j = 159992
titles[titles.Movie_Id.isin(good.iloc[j].split(" "))]["Name"]


1144     The Wedding Planner
2151         What Women Want
5316       Miss Congeniality
6286            Pretty Woman
6385              Sister Act
7233            Men of Honor
10358          Runaway Bride
11148      Maid in Manhattan
13049       Two Weeks Notice
15581     Sweet Home Alabama
Name: Name, dtype: object

In [15]:
def print_rule_title(rule):
    return titles[titles.Movie_Id == int(rule["to"])]["Name"].values


result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(" ")) & set(asr_df["from"].loc[A].split(" "))) == len(
        asr_df["from"].loc[A].split(" ")
    ):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Dirty Dancing', 'Pretty Woman'}


In [16]:
j = 130
pprint(titles[titles.Movie_Id.isin(good.iloc[j].split(" "))]["Name"])


5806                Raising Helen
10079    Six Feet Under: Season 2
14620         Shrek (Full-screen)
Name: Name, dtype: object


In [39]:
df.loc[(df.Cust_Id==130)&(df.Rating==5), ["Movie_Id"]].Movie_Id.to_list()

[1865,
 3456,
 3962,
 5515,
 6029,
 6428,
 8159,
 8327,
 8782,
 8784,
 11165,
 11242,
 11701,
 12084,
 12232,
 14482]

In [17]:
get_rule_title(asr_df.loc[315])


['The Patriot']
----------->
['The Green Mile']


In [20]:
asr_df

,from,to,confidence,support,lift
0,10042,10820,0.266215,0.0415732,4.11022
1,10042,10947,0.351478,0.0548881,2.42194
2,10042,11064,0.348135,0.0543661,2.3007
3,10042,11089,0.331616,0.0517864,2.48827
4,10042,11283,0.454613,0.0709941,2.20623
...,...,...,...,...,...
2018,11521,7057 16265 7230 14961 5582 2452,0.199233,0.0414848,4.63066
2019,11521,7057 7230 14961 9628 5582 2452,0.195898,0.0407902,4.63469
2020,14240,7057 16265 7230 14961 5582 2452,0.208372,0.0417112,4.84307
2021,14240,7057 7230 14961 9628 5582 2452,0.204569,0.0409498,4.83983


In [22]:
j = 21
result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(" ")) & set(asr_df["from"].loc[A].split(" "))) == len(
        asr_df["from"].loc[A].split(" ")
    ):
        result.append(print_rule_title(asr_df.loc[A])[0])
pprint(set(result))

{'Finding Nemo (Widescreen)',
 'Lord of the Rings: The Two Towers: Extended Edition',
 'Star Wars: Episode V: The Empire Strikes Back',
 'The Lord of the Rings: The Fellowship of the Ring: Extended Edition',
 'The Sixth Sense'}
